<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/dbpopulate_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3 as db
import numpy as np

In [2]:
#Import .py modules with processed data: readcsv contains Kaggle data, WebScrapingAtlantaEvents contains Discover Atlanta event data,
#and APICALL contains Budget Car Rental data
import LuxuryCarAPIMultipleDates

ModuleNotFoundError: No module named 'creds'

In [5]:
import WebScrapingAtlantaEvents

In [6]:
import readcsv

In [7]:
# Creates a blank db if one does not exist, otherwise opens connection to rentaldb, and drops the tables Kaggle, API, and Webscrape
conn = db.connect('rentaldb')
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS Kaggle")
c.execute("DROP TABLE IF EXISTS API")
c.execute("DROP TABLE IF EXISTS LuxuryAPI")
c.execute("DROP TABLE IF EXISTS AllCarsAPI")
c.execute("DROP TABLE IF EXISTS Webscrape")
c.execute("DROP TABLE IF EXISTS LargeEvents")

In [8]:
#Runs the sort_lux_df function from the readcsv import. This function reads the CarRentalDataV1 csv data into a dataframe
#sorts it based on location (Atlanta), then filters the vehciles to only show luxury brands/models.
csvdf = readcsv.sort_lux_df()
#This injects the sort_lux_df into the Kaggle table of the rentaldb 
csvdf.to_sql('Kaggle', conn, if_exists='replace')

#Runs the get_events function from the WebScrapingAtlantaEvents import. This function scrapes Discover Atlanta, 
#and formats the events
webdf = WebScrapingAtlantaEvents.get_events('2022-11-20', '2023-03-03')
#This injects the get_events data into the Webscrape table of the rentaldb 
webdf.to_sql('Webscrape', conn, if_exists='replace')

#Gets large venue events at the Mercedes-Benz stadium and State Farm Arena, and injects into the LargeEvents table of the rentaldb
large_venue_query = 'Location.str.contains("State Farm Arena|Mercedes-Benz Stadium")'
web_large_venue_df = WebScrapingAtlantaEvents.get_events('2022-11-20', '2023-03-03', query_string=large_venue_query)
web_large_venue_df.to_sql('LargeEvents', conn, if_exists='replace')

#Runs the get_bookings_multiple_dates function from LuxuryCarAPIMultipleDates. This function calls the API and requests the specified days of booking information.
#We will retrieve all cars and inject into AllCarsAPI, and luxury cars and inject into LuxuryAPI
all_cars_apidf = LuxuryCarAPIMultipleDates.get_bookings_multiple_dates('2022-11-20', '2023-03-03')
all_cars_apidf.to_sql('AllCarsAPI', conn, if_exists='replace')
all_cars_apidf.to_csv('AllCarsAPI.csv')
all_cars_apidf.to_pickle('AllCarsAPI.pkl')

luxuryapidf = all_cars_apidf.loc[all_cars_apidf['Category']=='Luxury'] 
luxuryapidf.to_sql('LuxuryAPI', conn, if_exists='replace')
luxuryapidf.to_csv('LuxuryAPI.csv')
luxuryapidf.to_pickle('LuxuryAPI.pkl')


In [13]:
#Verifying the tables are in the rentaldb
csvdf = pd.read_sql_query("SELECT * FROM Kaggle", conn)
luxuryapidf = pd.read_sql_query("SELECT * FROM LuxuryAPI", conn)
scrapedf = pd.read_sql_query("SELECT * FROM Webscrape", conn)
all_cars_apidf = pd.read_sql_query("SELECT * FROM AllCarsAPI", conn)
web_large_venue_df = pd.read_sql_query("SELECT * FROM LargeEvents", conn)

print('Kaggle Data')
display(csvdf.head())
print('All Car Data')
display(all_cars_apidf.head())
print('Luxury API Data')
display(luxuryapidf.tail(50))
print('Webscrape Data')
display(scrapedf.head())
print('Large Venue Events')
display(web_large_venue_df)

Kaggle Data


,index,fuelType,rating,renterTripsTaken,reviewCount,city,country,latitude,location.longitude,location.state,owner.id,daily_rate,make,model,type,year,airportcity,Vehicle_Name
0,41,GASOLINE,5.00,12,10,Grayson,US,33.906905,-83.960234,GA,1295660,175,Mercedes-Benz,S-Class,car,2015,Atlanta,Mercedes-Benz S-Class
1,42,GASOLINE,5.00,2,2,Atlanta,US,33.891295,-84.453929,GA,12796797,200,BMW,7 Series,car,2018,Atlanta,BMW 7 Series
2,50,GASOLINE,4.96,152,140,Atlanta,US,33.927684,-84.343055,GA,511103,155,Chevrolet,Corvette Stingray,car,2014,Atlanta,Chevrolet Corvette Stingray
3,54,GASOLINE,5.00,23,21,Atlanta,US,33.740676,-84.358236,GA,8752492,209,Audi,RS 7,car,2014,Atlanta,Audi RS 7
4,57,ELECTRIC,4.88,18,16,Duluth,US,33.970512,-84.161462,GA,8609349,159,Tesla,Model S,car,2018,Atlanta,Tesla Model S


All Car Data


,index,Vehicle_id,Vehicle_Name,Category,Transmission,Base_Price,Price,Date_Pickup,Date_Dropoff
0,281,658131758,Nissan Quest,Full-size,Automatic,238.24,246.85,2022-11-20,2022-11-21
1,280,658184283,Nissan Quest,Full-size,Automatic,236.96,245.53,2022-11-20,2022-11-21
2,277,659534526,Nissan Quest,Mini,Automatic,226.76,234.96,2022-11-20,2022-11-21
3,265,642897531,Nissan Quest,Full-size,Automatic,225.49,233.64,2022-11-20,2022-11-21
4,276,643094521,Nissan Quest,Mini,Automatic,225.49,233.64,2022-11-20,2022-11-21


Luxury API Data


,index,Vehicle_id,Vehicle_Name,Category,Transmission,Base_Price,Price,Date_Pickup,Date_Dropoff
2554,217,723428946,BMW 5 Series,Luxury,Automatic,191.12,198.03,2023-03-01,2023-03-02
2555,248,636479888,Cadillac XTS,Luxury,Automatic,190.95,197.85,2023-03-01,2023-03-02
2556,216,646119413,Cadillac XTS,Luxury,Automatic,185.35,192.05,2023-03-01,2023-03-02
2557,227,644901168,Cadillac XTS,Luxury,Automatic,180.76,187.29,2023-03-01,2023-03-02
2558,251,658137488,Chrysler 300,Luxury,Automatic,175.83,182.19,2023-03-01,2023-03-02
2559,253,658190013,Chrysler 300,Luxury,Automatic,174.59,180.90,2023-03-01,2023-03-02
2560,235,636492688,Cadillac XTS,Luxury,Automatic,174.15,180.45,2023-03-01,2023-03-02
2561,223,644861493,Cadillac XTS,Luxury,Automatic,169.48,175.61,2023-03-01,2023-03-02
2562,219,646151293,Cadillac XTS,Luxury,Automatic,168.54,174.63,2023-03-01,2023-03-02
2563,115,659534556,Chrysler 300,Luxury,Automatic,164.73,170.68,2023-03-01,2023-03-02


Webscrape Data


,index,Event,Dates,Location,Description
0,1679,“The Tempest”,2022-11-21,Shakespeare Tavern Playhouse,"Shipwrecked after a violent storm, little do t..."
1,1680,2022 Georgia Festival of Trees,2022-11-21,Georgia World Congress Center,Kick off your holiday season with the Georgia ...
2,1681,FIFA World Cup 2022™ Watch Parties,2022-11-21,Multiple Locations,"Throughout the 2022 FIFA World Cup™, Atlanta U..."
3,1682,Mattie Lou O’Kelley: The Long Journey to Success,2022-11-21,Oglethorpe University Museum of Art,Like most female artists of the early 20th cen...
4,1683,Aldabra Tortoise Wild Encounter,2022-11-21,Zoo Atlanta,Meet the ancient giants of Aldabra Atoll. See ...


Large Venue Events


,index,Event,Dates,Location,Description
0,1728,ATEEZ – The Fellowship: Break The Wall Tour,2022-11-22,State Farm Arena,K-pop giants ATEEZ have announced a monumental...
1,1771,Atlanta Hawks vs. Sacramento Kings,2022-11-23,State Farm Arena,Come cheer on the Hawks as they take on the Sa...
2,2023,Atlanta Hawks vs. Miami Heat,2022-11-27,State Farm Arena,Come cheer on the Hawks as they take on the Mi...
3,2202,Rod Wave – Beautiful Mind Tour,2022-12-01,State Farm Arena,Platinum-selling rapper Rod Wave has announced...
4,2236,Atlanta Hawks vs. Denver Nuggets,2022-12-02,State Farm Arena,Come cheer on the Hawks!
5,2255,Disney On Ice: Frozen & Encanto,2022-12-02,State Farm Arena,"For the first time, Disney On Ice invites fami..."
6,2280,2022 SEC Football Championship,2022-12-03,Mercedes-Benz Stadium,The 2022 SEC Championship Game is on course to...
7,2370,Atlanta Falcons vs. Pittsburgh Steelers – Dec. 4,2022-12-04,Mercedes-Benz Stadium,Come watch the Atlanta Falcons take on the Pit...
8,2372,Atlanta R&B Music Experience,2022-12-04,State Farm Arena,It was announced today that ‘Atlanta R&B Music...
9,2390,Atlanta Hawks vs. Oklahoma City Thunder,2022-12-05,State Farm Arena,Come cheer on the Hawks as they take on the Ok...
